In [136]:
import io
import base64
import requests
import json 
from PIL import Image
import hashlib

In [137]:
def convert_image(image_path):
    with Image.open(image_path) as img:
        buffer = io.BytesIO()
        if img.format not in ['PNG', 'JPEG', 'GIF', 'WEBP']:
            img.save(buffer, format="PNG")
            extension = "png"
        else:
            img.save(buffer, format=img.format)
            extension = img.format.lower()
        
        processed_image_path = f"processed_image.{extension}"
        with open(processed_image_path, "wb") as out_file:
            out_file.write(buffer.getvalue())
            
        buffer.seek(0)  
        return buffer.getvalue(),processed_image_path, extension
    

def calculate_checksum(image_data):
    return hashlib.md5(image_data).hexdigest()

In [148]:
# Path to your image file
image_path = "img6.jpg"

# Convert the image
image_data, image_path, extension = convert_image("img6.jpg")

checksum = calculate_checksum(image_data)

print("Checksum:", checksum)

encoded_image = base64.b64encode(image_data).decode('utf-8')
     
endpoint = f"https://plvf1e97ll.execute-api.us-east-1.amazonaws.com/production/deteccion" 
    
headers = {
        "Content-Type": "application/json"
    } 

inputRoot = {
    "Checksum": checksum,
    "Extension": extension,
    "Image": encoded_image 
} 

dumbs = json.dumps(inputRoot)

response = requests.post(endpoint, headers=headers, data=dumbs)
    
processed_image_path = "processed_image.json"
with open(processed_image_path, "w") as out_file:
    json.dump(inputRoot, out_file, indent=4)
            
if response.status_code == 200:
    print("Respuesta del servidor:", response.json())
else:
    print("Error en la solicitud:", response.status_code, response.text)
    
print("Fin de la ejecución")

Checksum: 649a13b5ed113c097a45b29f9ab726c1
